In [1]:
import os
import cv2
from tqdm import tqdm
import numpy as np
import math
import scipy
import open3d as o3d
import read_raw_file as RRF
import preprocess as PRE
import copy


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
path = r'D:\StageE23\Data\Ete_2022\Participant02\BD\Libre\Prise01'
save_path1 = path + '/intensity_removed_bg/'
save_path2 = path + '/xyz_removed_bg/'
intensity_path = path + '/intensity/'
xyz_path = path + '/xyz/'

os.makedirs(save_path1, exist_ok=True)
os.makedirs(save_path2, exist_ok=True)

for filename in tqdm(os.listdir(intensity_path)):
        index_I = filename.find('_I') + 1
        xyz_file = filename[:index_I] + 'XYZ_' + filename[index_I+2:-4] + '.raw'
        image_path = os.path.join(intensity_path, filename)
        pc_path = os.path.join(xyz_path, xyz_file)
        xyz = PRE.read_single_xyz_raw_file(pc_path)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        removed_bg_points, image_no_bg = PRE.remove_bg(xyz, image)
        
        save_image_path = os.path.join(save_path1, filename)
        cv2.imwrite(save_image_path, image_no_bg)

        pc_removed_bg = o3d.geometry.PointCloud()
        pc_removed_bg.points = o3d.utility.Vector3dVector(removed_bg_points)
        cl, ind = pc_removed_bg.remove_statistical_outlier(nb_neighbors=40,
                                                std_ratio=1.6)
        inlier_removed_bg = pc_removed_bg.select_by_index(ind)

        save_xyz_path = os.path.join(save_path2, xyz_file[:-4] + '.ply')
        o3d.io.write_point_cloud(save_xyz_path, inlier_removed_bg)

100%|██████████| 92/92 [02:01<00:00,  1.32s/it]


In [3]:
pc0 = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant02\BD\Libre\Prise01\xyz_removed_bg\BD_libr_01_001034_XYZ_0.ply')
pc1 = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant02\BD\Libre\Prise01\xyz_removed_bg\BD_libr_01_001035_XYZ_1.ply')
pc40 = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant02\BD\Libre\Prise01\xyz_removed_bg\BD_libr_01_001082_XYZ_40.ply')
pc50 = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant02\BD\Libre\Prise01\xyz_removed_bg\BD_libr_01_001094_XYZ_50.ply')
pc72 = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant02\BD\Libre\Prise01\xyz_removed_bg\BD_libr_01_001120_XYZ_72.ply')

o3d.visualization.draw_geometries([pc0])

In [4]:
def crop_pc(pc):
    pc_array = np.asarray(pc.points)
    print(pc_array)
 
    legs = np.where(pc_array[:,0] < -300)
    head = np.where(pc_array[:,0] > np.max(pc_array[:,0])-300)
    armR = np.where(pc_array[:,1] > np.max(pc_array[:,1])-90)
    armL = np.where(pc_array[:,1] < np.min(pc_array[:,1])+90)

    mask = np.ones((pc_array.shape[0],3), dtype=bool)
    mask[legs] = False
    mask[head] = False
    mask[armR] = False
    mask[armL] = False
    #mask.reshape((500123, 3))

    back = np.array(pc_array[mask], dtype=np.float64)

    pc = np.array(back.reshape((back.shape[0]//3, 3)))

    pc_cropped = o3d.geometry.PointCloud()
    pc_cropped.points = o3d.utility.Vector3dVector(pc)
    
    return pc_cropped

pc_center = crop_pc(pc0)
o3d.visualization.draw_geometries([pc_center])


[[-185.04086304 -203.23529053 1796.91711426]
 [-185.97872925 -203.29910278 1797.43640137]
 [-186.9108429  -203.35600281 1797.89440918]
 ...
 [-230.64271545  203.10369873 1842.90161133]
 [-235.64268494  203.52252197 1846.41516113]
 [-236.64251709  203.60415649 1847.09814453]]


In [5]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [18]:
body0 = crop_pc(pc0)
body1 = crop_pc(pc1)
body40 = crop_pc(pc40)
body50 = crop_pc(pc50)
body72 = crop_pc(pc72)

trans_init = np.identity(4)

icp1 = o3d.pipelines.registration.registration_icp(body0, body1, 10, trans_init, o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(icp1.transformation)

icp50 = o3d.pipelines.registration.registration_icp(body40, body50, 10, trans_init, o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(icp50.transformation)

draw_registration_result(body40, body50, icp50.transformation)

[[-185.04086304 -203.23529053 1796.91711426]
 [-185.97872925 -203.29910278 1797.43640137]
 [-186.9108429  -203.35600281 1797.89440918]
 ...
 [-230.64271545  203.10369873 1842.90161133]
 [-235.64268494  203.52252197 1846.41516113]
 [-236.64251709  203.60415649 1847.09814453]]
[[-186.86268616 -203.30361938 1797.43127441]
 [-187.8059082  -203.37220764 1797.99243164]
 [-188.70266724 -203.39013672 1798.10559082]
 ...
 [-236.66604614  203.62440491 1847.28186035]
 [-237.67375183  203.7121582  1848.01965332]
 [-238.79000854  203.891922   1849.59216309]]
[[ 354.95379639 -204.87136841 2043.01904297]
 [ 354.03741455 -204.92196655 2043.60705566]
 [ 352.96954346 -204.88502502 2043.32165527]
 ...
 [ 481.1847229   259.90771484 1883.74841309]
 [ 482.19720459  260.88504028 1883.8815918 ]
 [ 480.70889282  260.58505249 1881.8260498 ]]
[[-318.1159668  -206.51820374 1825.18615723]
 [-319.02301025 -206.5242157  1825.16357422]
 [-294.55819702 -205.47015381 1825.79199219]
 ...
 [ 402.31640625  386.63476562 18

In [16]:
RST = icp50.transformation
T = RST[0:3, 3]
print(T)
len_T = np.linalg.norm(T)
print(f'Distance de translation : {len_T} mm')
RS = RST[0:3, 0:3]
print(RS)
S = np.zeros((3,3))
S[0,0] = np.linalg.norm(RS[:,0])
S[1,1] = np.linalg.norm(RS[:,1])
S[2,2] = np.linalg.norm(RS[:,2])
print(S)
R = np.matmul(RS, np.linalg.inv(S)) #S est la matrice identité, donc R=RS, aucun scaling
print(R)

[ -8.38374302 125.10506713  -1.46606492]
Distance de translation : 125.39423557208771 mm
[[ 0.99497345 -0.10012251  0.00182253]
 [ 0.10003751  0.99297978 -0.06311614]
 [ 0.00450961  0.06298121  0.99800452]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[ 0.99497345 -0.10012251  0.00182253]
 [ 0.10003751  0.99297978 -0.06311614]
 [ 0.00450961  0.06298121  0.99800452]]


In [12]:
Tr = np.trace(R)
cos_rot = (Tr-1)/2
angle_rot = np.degrees(np.arccos(cos_rot))
print(angle_rot)

axis_rot = np.multiply(1/np.sqrt((3-Tr)*(1+Tr)), np.array([R[2,1]-R[1,2], R[0,2]-R[2,0], R[1,0]-R[0,1]]))
print(axis_rot)

6.793527411413739
[ 0.53299279 -0.01135781  0.84604355]


In [13]:
def IsRotationMatrix(R):
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

def RotationMatrixToEulerAngles(R):
    assert(IsRotationMatrix(R))

    sy = math.sqrt(R[0,0] * R[0,0] + R[1,0] * R[1,0])

    singular = sy < 1e-6

    if not singular:
        y = math.atan2(R[2,1], R[2,2])
        x = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else:
        y = math.atan2(-R[1,2], R[1,1])
        x = math.atan2(-R[2,0], R[1,1])
        z = 0

    return np.array([np.degrees(x),np.degrees(y),np.degrees(z)])

print(RotationMatrixToEulerAngles(R))

[-0.2583823   3.61098395  5.74138883]
